# This notebook serves as walkthrough for planning an experiment for creation through the OT2.
### The following modules are used and should be in the same directory as this notebook: 
* **CreateSamples** is responsible for calculating sample information, which includes component weight fractions and stock volumes
* **OT2Commands** is responsible for setting up information to be interpretted and executed by opentrons **BASED ON THE LATEST API 2.3 and above**
* **OT2Graphing** contains graphing tools to help visualize and explore parameter spaces.

In [109]:
from Plan import CreateSamples
from Prepare import OT2Commands as ALH
from Prepare import OT2Graphing as ographing
from opentrons import simulate, execute, protocol_api
import importlib
import pandas as pd

In [110]:
importlib.reload(ALH)

<module 'Prepare.OT2Commands' from 'C:\\Users\\lacho\\Documents\\OT2-DOE\\OT2_DOE\\Prepare\\OT2Commands.py'>

### Step 1: Set up the experiment dictionary.
The first step to planning an experiment is to load the experiment variables and inputs from a csv file. **Within the Testing Plans Folder you should find a templete of a testing plan (open in Excel for easier viewing).**

Currently the info is split up between two sections: Variables for creating sample spaces and variables for OT2 commands. You only need to specify for the sections you will use.The experiment dictionary consist of keys being the variable name and the value being the variables value. Keep in mind the order of listed variables assumptions should be the index of one listed variable refers to the same index of a related variable.

Besides the plan, if planning on creating a sample space you will need to:
- Update/verify the csv chemical database with/for component information.
- Provide the path of the updated chemical csv for plan. 

In [111]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 2CE9-CE47

 Directory of C:\Users\lacho\Documents\OT2-DOE\OT2_DOE

11/04/2022  03:40 PM    <DIR>          .
11/04/2022  03:40 PM    <DIR>          ..
11/04/2022  01:11 PM    <DIR>          .idea
11/04/2022  01:16 PM    <DIR>          .ipynb_checkpoints
11/04/2022  01:11 PM                 0 __init__.py
11/04/2022  03:04 PM             2,405 Chemical Database.csv
11/04/2022  01:11 PM    <DIR>          Custom Labware
11/04/2022  01:11 PM             1,417 Example_Opentrons_Protocol.csv
11/04/2022  01:11 PM            99,841 GP Regression.ipynb
11/04/2022  01:11 PM           406,871 OT2_Samples_Formulation-Copy1.ipynb
11/04/2022  01:11 PM           713,985 pipettes_and_volumes.ipynb
11/04/2022  01:18 PM    <DIR>          Plan
11/04/2022  03:38 PM           197,480 Plan and Prepare Notebook.ipynb
11/04/2022  01:11 PM         1,672,609 Plan and Prepare Notebook-B21-robot.ipynb
11/04/2022  01:11 PM           188,981 Plate Reader Notebook

In [112]:
path = "./Z2M246I_Protocol.csv"
chem_path = r"Chemical Database.csv"
plan = CreateSamples.get_experiment_plan(path, chem_path)

### Step 2: Select and Create Sampling Space
The sampling method will depend on the type of sample units, the actual called method (i.e. lattic, non-clustering random..etc) and whether you are providing arguments from the plan in step 1 or importing sample information from elsewhere. 

***Note:*** All information will be stored as a dataframe
 
You can utilize the built in functions in functions to create sample concentration spaces as follows:
- concentration_from_csv: Import csv with headers being component name and concentration unit.
- concentration_from_excel: Import from excel with headers being component name and concentration unit.
- concentration_from_linspace: Pull from plan the linspaces to use lattice or random sampling. Optional: Unity filter allows for last indexed component to be used a completing component. 
- concentration_from_list_samplewise: 
- concentration_from_list_componentwise:
 
**The ONE requirement if you plan to use any of the module functions is you adhere to strict naming convention. This allows the functions to identify key values to use to calculate and determine specific informaiton (i.e. mass, volume to pipette). **

The following conventions are:
- component concentration columns: Should have the term "concentration" and the concentration unit name (i.e. DPPC Concentration Molarity). 
-

In [113]:
# concentration_df = CreateSamples.concentration_from_linspace(plan['Component Shorthand Names'], 
#                                                              plan['Component Concentration Linspaces [min, max, n]'],
#                                                              plan['Component Concentration Units'],
#                                                              unity_filter=True)        
# concentration_df

concentration_df = CreateSamples.concentration_from_list_componentwise_grid(plan['Component Shorthand Names'],
                                                                            plan['Component Concentration List'],
                                                                            plan['Component Concentration Units'])
concentration_df

,HEPES concentration molarity,Z2M246I concentration molarity,HAuCl4 concentration molarity
0,0.001,0.000040,0.000040
1,0.001,0.000040,0.000068
2,0.001,0.000040,0.000117
3,0.001,0.000040,0.000200
4,0.001,0.000089,0.000040
5,0.001,0.000089,0.000068
6,0.001,0.000089,0.000117
7,0.001,0.000089,0.000200
8,0.001,0.000200,0.000040
9,0.001,0.000200,0.000068


### Step 3: Create Stock Dictionary
The stock dictionary will hold the stock information which will be used to calculate for stock volumes for the OT2.
The stock dictionary function will take 3 main sets of information (all formatted as lists, matched by index). 
- Stock Names: Stock names are to be formatted as solute1-solute2...soluten-solvent-stock i.e. NaCl-water-stock.
    - The solutes and solvents are expected to be in the chemical dictionary provided in step 1 as information such as density and molecular weight will be used in volume calculations.
- Stock Concentration Units: Any conc units supported (see function).
- Stock Concentrations: Value of stock conc.

In [114]:
stock_dict = CreateSamples.stock_dictionary(plan['Stock Names'], 
                                                    plan['Stock Concentration Units'], 
                                                    plan['Stock Concentrations'])
stock_dict

{'HEPES-water-stock': {'solutes': ['HEPES'],
  'solvents': 'water',
  'unit': 'molarity',
  'concentration': 0.004183780862202961,
  'Density (g/mL)': nan,
  'Common Solvent': 'Mixture'},
 'Z2M246I-water-stock': {'solutes': ['Z2M246I'],
  'solvents': 'water',
  'unit': 'molarity',
  'concentration': 0.0005383369330453564,
  'Density (g/mL)': nan,
  'Common Solvent': 'Mixture'},
 'HAuCl4-water-stock': {'solutes': ['HAuCl4'],
  'solvents': 'water',
  'unit': 'molarity',
  'concentration': 0.002934165219694517,
  'Density (g/mL)': nan,
  'Common Solvent': 'Mixture'}}

### Step 3: Calculate Component Amounts Mass And Volumes
The end goal in this component planning is to calculate for stock volumes, this requires knowledge on the component mass/volumes.

For now there exist one general function *determine_component_amounts* which based on the total amount (either volume or mass) and component conc units will use the appropiate function to calculate for component mass and volume amounts. 

Since this step uses information from the chemical database (density and mw), if that information is missing or not avaible then the resulting values will return as nan. To remove these and replace with zero in cases where the component mass/volume is negligible use the argument within the function to specify a fill amount of nan_fill_value = 0.

In [115]:
complete_amounts_added = CreateSamples.determine_component_amounts(plan, concentration_df, nan_fill_value=0)
complete_amounts_added

You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units


,HEPES concentration molarity,Z2M246I concentration molarity,HAuCl4 concentration molarity,HEPES amount mass g,HEPES amount volume mL,Z2M246I amount mass g,Z2M246I amount volume mL,HAuCl4 amount mass g,HAuCl4 amount volume mL
0,0.001,0.000040,0.000040,0.000071,0.0,0.000011,0.0,0.000004,0.0
1,0.001,0.000040,0.000068,0.000071,0.0,0.000011,0.0,0.000007,0.0
2,0.001,0.000040,0.000117,0.000071,0.0,0.000011,0.0,0.000012,0.0
3,0.001,0.000040,0.000200,0.000071,0.0,0.000011,0.0,0.000020,0.0
4,0.001,0.000089,0.000040,0.000071,0.0,0.000025,0.0,0.000004,0.0
5,0.001,0.000089,0.000068,0.000071,0.0,0.000025,0.0,0.000007,0.0
6,0.001,0.000089,0.000117,0.000071,0.0,0.000025,0.0,0.000012,0.0
7,0.001,0.000089,0.000200,0.000071,0.0,0.000025,0.0,0.000020,0.0
8,0.001,0.000200,0.000040,0.000071,0.0,0.000056,0.0,0.000004,0.0
9,0.001,0.000200,0.000068,0.000071,0.0,0.000056,0.0,0.000007,0.0


### Step 4: Calculate Volumes of Stocks
From the mass/volume of individual chemical components we calculauted along with the concentration and identities of the provided stocks, we are now able to combine those two pieces of information and obtain the volume of stock needed. 

The general function to use Create.calculate_stock_volumes_from_component_concs will autoamatically use the component information either volume or stock depending on the stock which contains the component and its stock unit. The stock unit essentially determines the pathway of whether component mass or volume will be utilized.See ______ for a map of how specific component concentration untis and stock units pair together to calculate for info. 

Note: This will calculate stock volumes not taking into account any interdepdencies such as common solvents or missing volume to complete a desired sample volume. The next step will have a few cases for which specfic functions can calculate and adjust stock volumes. 

In [116]:
stock_volumes_prelim = CreateSamples.calculate_stock_volumes_from_component_concs(plan, complete_amounts_added, stock_dict)
stock_volumes_prelim

,HEPES concentration molarity,Z2M246I concentration molarity,HAuCl4 concentration molarity,HEPES amount mass g,HEPES amount volume mL,Z2M246I amount mass g,Z2M246I amount volume mL,HAuCl4 amount mass g,HAuCl4 amount volume mL,HEPES-water-stock amount volume mL,Z2M246I-water-stock amount volume mL,HAuCl4-water-stock amount volume mL
0,0.001,0.000040,0.000040,0.000071,0.0,0.000011,0.0,0.000004,0.0,0.071705,0.022291,0.004090
1,0.001,0.000040,0.000068,0.000071,0.0,0.000011,0.0,0.000007,0.0,0.071705,0.022291,0.006993
2,0.001,0.000040,0.000117,0.000071,0.0,0.000011,0.0,0.000012,0.0,0.071705,0.022291,0.011958
3,0.001,0.000040,0.000200,0.000071,0.0,0.000011,0.0,0.000020,0.0,0.071705,0.022291,0.020449
4,0.001,0.000089,0.000040,0.000071,0.0,0.000025,0.0,0.000004,0.0,0.071705,0.049844,0.004090
5,0.001,0.000089,0.000068,0.000071,0.0,0.000025,0.0,0.000007,0.0,0.071705,0.049844,0.006993
6,0.001,0.000089,0.000117,0.000071,0.0,0.000025,0.0,0.000012,0.0,0.071705,0.049844,0.011958
7,0.001,0.000089,0.000200,0.000071,0.0,0.000025,0.0,0.000020,0.0,0.071705,0.049844,0.020449
8,0.001,0.000200,0.000040,0.000071,0.0,0.000056,0.0,0.000004,0.0,0.071705,0.111454,0.004090
9,0.001,0.000200,0.000068,0.000071,0.0,0.000056,0.0,0.000007,0.0,0.071705,0.111454,0.006993


### Step 4a: Adjust/Add Stock Volumes

Case 1: Common solvents, in this case solvents are shared between more than one stock the stock needs to be taken into account if provided as component in the first place. For example if ethanol is present in 2 stocks and ethanol must be a specfic wtf then we must take into account the ethanol added when adding those two stocks. The case may arise where you do not specify the concentration of the common solvent and in that case we do not care about the final concentration of common solvent so no need to track for it. The function you would use is **Create.calculate_common_solvent_residual_volumes()**

Case 2: Filling case, in this case after calculating each stock volume for a sample it may need to be diluted/have the total sample volume completed by a component. The function to use is **Create.complete_missing_volume_with_commmon_solvent()** which you can specify the name of a new stock along with the total completing volume and/or if not provided will automatically look for a single solvent stock and use that to fill.

Case X: Other?

***NOTE: ALWAYS CONVERT THE FINAL mL TO uL AS OT2 NATIVELY READS ALL NUMBERS AS uL USE THE CreateSamples.convert_mL_to_uL() FUNCTION ***

In [117]:
stock_dict

{'HEPES-water-stock': {'solutes': ['HEPES'],
  'solvents': 'water',
  'unit': 'molarity',
  'concentration': 0.004183780862202961,
  'Density (g/mL)': nan,
  'Common Solvent': 'Mixture'},
 'Z2M246I-water-stock': {'solutes': ['Z2M246I'],
  'solvents': 'water',
  'unit': 'molarity',
  'concentration': 0.0005383369330453564,
  'Density (g/mL)': nan,
  'Common Solvent': 'Mixture'},
 'HAuCl4-water-stock': {'solutes': ['HAuCl4'],
  'solvents': 'water',
  'unit': 'molarity',
  'concentration': 0.002934165219694517,
  'Density (g/mL)': nan,
  'Common Solvent': 'Mixture'}}

In [118]:
stock_volumes_completed = CreateSamples.complete_missing_volume_with_commmon_solvent(plan['Sample Amount'],stock_volumes_prelim, stock_dict, 'water')
complete_df = CreateSamples.convert_mL_to_uL(stock_volumes_completed)
# complete_df[complete_df['ethanol concentration wtf'] == 0.1]

In [119]:
complete_df

,HEPES concentration molarity,Z2M246I concentration molarity,HAuCl4 concentration molarity,HEPES amount mass g,HEPES amount volume uL,Z2M246I amount mass g,Z2M246I amount volume uL,HAuCl4 amount mass g,HAuCl4 amount volume uL,HEPES-water-stock amount volume uL,Z2M246I-water-stock amount volume uL,HAuCl4-water-stock amount volume uL,water-stock volume uL
0,0.001,0.000040,0.000040,0.000071,0.0,0.000011,0.0,0.000004,0.0,71.705476,22.290873,4.089749,201.913902
1,0.001,0.000040,0.000068,0.000071,0.0,0.000011,0.0,0.000007,0.0,71.705476,22.290873,6.993373,199.010278
2,0.001,0.000040,0.000117,0.000071,0.0,0.000011,0.0,0.000012,0.0,71.705476,22.290873,11.958499,194.045152
3,0.001,0.000040,0.000200,0.000071,0.0,0.000011,0.0,0.000020,0.0,71.705476,22.290873,20.448746,185.554905
4,0.001,0.000089,0.000040,0.000071,0.0,0.000025,0.0,0.000004,0.0,71.705476,49.843906,4.089749,174.360868
5,0.001,0.000089,0.000068,0.000071,0.0,0.000025,0.0,0.000007,0.0,71.705476,49.843906,6.993373,171.457244
6,0.001,0.000089,0.000117,0.000071,0.0,0.000025,0.0,0.000012,0.0,71.705476,49.843906,11.958499,166.492118
7,0.001,0.000089,0.000200,0.000071,0.0,0.000025,0.0,0.000020,0.0,71.705476,49.843906,20.448746,158.001871
8,0.001,0.000200,0.000040,0.000071,0.0,0.000056,0.0,0.000004,0.0,71.705476,111.454363,4.089749,112.750411
9,0.001,0.000200,0.000068,0.000071,0.0,0.000056,0.0,0.000007,0.0,71.705476,111.454363,6.993373,109.846788


### Step 5: Apply Filters or Other Selection Criteria to Stock Volumes

**Filters**: The types of filters can be added easily through dataframe logic. For example, ones in the CreateSamples module:
   * *filter_general_min_pipette_restriction* = Given the lower of the pipettes you will use to synthesize the samples, will remove anything not in range or zero. 
   * *filter_total_volume_restriction* = Restrict total volume, mostly based on labware constraints. Column selection based on "total". 
   * *filter_general_max_restriction* = provide max value and column name 
   
**Other Selection Critera**
   * *.multiplydf(n, df)* = Will multiply a data frame by n number of times. Useful when making one sample repeatadly.

In [120]:
complete_df_filtered = CreateSamples.filter_general_min_pipette_restriction(complete_df, 2)
complete_df_filtered = CreateSamples.filter_total_volume_restriction(complete_df_filtered,1600)
complete_df_filtered = CreateSamples.filter_general_max_restriction(complete_df_filtered,1000, 'water-stock volume uL')
complete_df_filtered.reset_index(inplace=True,drop=True)

complete_df_filtered.shape

(16, 14)

In [121]:
complete_df_filtered = CreateSamples.filter_general_max_restriction(complete_df_filtered,1000, 'water-stock volume uL')
complete_df_filtered.reset_index(inplace=True,drop=True)



# complete_df_filtered = complete_df_filtered[complete_df_filtered['ethanol concentration wtf']>=0.6]
# complete_df_filtered = complete_df_filtered[complete_df_filtered['ODE concentration wtf']>0.00067]
# CreateSamples.isolate_common_column(complete_df_filtered, 'stock')

# complete_df_filtered.reset_index(inplace=True, drop=True)
# # len(complete_df_filtered)
complete_df_filtered.reset_index(inplace=True,drop=True)
complete_df_filtered

,HEPES concentration molarity,Z2M246I concentration molarity,HAuCl4 concentration molarity,HEPES amount mass g,HEPES amount volume uL,Z2M246I amount mass g,Z2M246I amount volume uL,HAuCl4 amount mass g,HAuCl4 amount volume uL,HEPES-water-stock amount volume uL,Z2M246I-water-stock amount volume uL,HAuCl4-water-stock amount volume uL,water-stock volume uL,Total Volume
0,0.001,0.000040,0.000040,0.000071,0.0,0.000011,0.0,0.000004,0.0,71.705476,22.290873,4.089749,201.913902,300.0
1,0.001,0.000040,0.000068,0.000071,0.0,0.000011,0.0,0.000007,0.0,71.705476,22.290873,6.993373,199.010278,300.0
2,0.001,0.000040,0.000117,0.000071,0.0,0.000011,0.0,0.000012,0.0,71.705476,22.290873,11.958499,194.045152,300.0
3,0.001,0.000040,0.000200,0.000071,0.0,0.000011,0.0,0.000020,0.0,71.705476,22.290873,20.448746,185.554905,300.0
4,0.001,0.000089,0.000040,0.000071,0.0,0.000025,0.0,0.000004,0.0,71.705476,49.843906,4.089749,174.360868,300.0
5,0.001,0.000089,0.000068,0.000071,0.0,0.000025,0.0,0.000007,0.0,71.705476,49.843906,6.993373,171.457244,300.0
6,0.001,0.000089,0.000117,0.000071,0.0,0.000025,0.0,0.000012,0.0,71.705476,49.843906,11.958499,166.492118,300.0
7,0.001,0.000089,0.000200,0.000071,0.0,0.000025,0.0,0.000020,0.0,71.705476,49.843906,20.448746,158.001871,300.0
8,0.001,0.000200,0.000040,0.000071,0.0,0.000056,0.0,0.000004,0.0,71.705476,111.454363,4.089749,112.750411,300.0
9,0.001,0.000200,0.000068,0.000071,0.0,0.000056,0.0,0.000007,0.0,71.705476,111.454363,6.993373,109.846788,300.0


### Step 6 (Optional): Calculate Stock Prep Information

We can calculate stock information to make it easier to prepare these stock. This process if semi-general, currently supporting a few stock concentration units = mgperml, volf, wtf, molf and molarity. All of these require the basis of volume as that is what is driving how much sample we make for the OT2. 

The logic behind this relies on the notation of stocks = solute1-solute2-....solvent. Using notations allows for systematic handling of solutes and solvent determining what is needed based on information from a excel based Chemical Database.

*Reccomended*: Use at the minimum the default buffering (extra stock being made) of 40 percent to ensure you do not aspirate any air by mistake.

BROKEN FIXING

In [122]:
# stock_volumes = CreateSamples.isolate_common_column(complete_df_filtered, 'stock')
# chem_database_path = r"Chemical Database.xlsx"
# stock_prep_df = CreateSamples.calculate_stock_prep_df(plan, stock_volumes, chem_database_path, buffer_pct=50)
# stock_prep_df
stock_volumes = CreateSamples.isolate_common_column(complete_df_filtered,'stock').sum(axis=0)/1000
stock_volumes

HEPES-water-stock amount volume uL      1.147288
Z2M246I-water-stock amount volume uL    0.734357
HAuCl4-water-stock amount volume uL     0.173961
water-stock volume uL                   2.744394
dtype: float64

### Step 8: Simulate/Execute
The main information needed to create commands for the OT2 is the position of the stocks and the volumes in respect to each well you would like to move the stock to. All volumes in sent to the OT2 will be interpretted as microliters (uL). 

* Pipetting Strategy 1: Add each stock to all wells then move onto next stock. The upside to this is the simplicity of the written protocol. It has a downside of using more tips. It can allow for evaporation or other time sensitve process to occur in the time between stock additions. This is referred to component wise pipetting.

* Pipetting Strategy 2: Adding all stocks to a ONE well then moving on, this is referred to as sample wise pipetting. 

### Step 8a Optional: Load custom labware dictionary (Remote Testing)
* Provide the path to the directory holding all custom labware. This directory should have custom labware .json files you have previously made and tested, read more here: https://support.opentrons.com/en/articles/3136504-creating-custom-labware-definitions
* The reason we provide this is when working on a device that is not connected to the OT2's Jupyter notebook there is no way to natively use custom labware. So we create a dictionary of custom labware so we can later load into our protocol to primarily simualte and test protocols for execution later once connnected to the OT2's notebook.
    * When using custom labware on the OT2's notebook it pulls from a folder labeled "labware", which is something built into the Opentrons hardware. It has not been tested if the custom labware dictionary will superceed this directory of custom labware if used on the OT2.
    * The custom labware folder should contain everything the OT2 labware folder contains: ADD GDRIVE link to customize
    
* **NOTE: Any function using the custom_labware_dict, the argument is optional so can remove and should receive same output IF working on OT2 computer.**

Just noted: transfer_info will fail if transfer and distribute used together, well not fail jsut need to concat

In [123]:
labware_dir_path = r"Custom Labware"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)
custom_labware_dict.keys()

dict_keys(['Custom Labware\\20mlscintillationsmall_12_wellplate_18000ul\\20mlscintillationsmall_12_wellplate_18000ul', 'Custom Labware\\20mlscintillation_12_wellplate_18000ul\\20mlscintillation_12_wellplate_18000ul', 'Custom Labware\\abgeneshort_96_wellplate_2200ul\\abgeneshort_96_wellplate_2200ul', 'Custom Labware\\abgene_96_wellplate_2200ul\\abgene_96_wellplate_2200ul', 'Custom Labware\\agilent_6_reservoir_47000ul\\agilent_6_reservoir_47000ul', 'Custom Labware\\bbgene_96_wellplate_2200ul\\bbgene_96_wellplate_2200ul', 'Custom Labware\\discher_24_wellplate_7400ul\\discher_24_wellplate_7400ul', 'Custom Labware\\falcon_48_wellplate_1500ul\\falcon_48_wellplate_1500ul', 'Custom Labware\\fischer_24_wellplate_7400ul\\fischer_24_wellplate_7400ul\\fischer_24_wellplate_7400ul', 'Custom Labware\\fisherbrand_2_wellplate_100000ul\\fisherbrand_2_wellplate_100000ul', 'Custom Labware\\helmaanalyticsquartz_96_wellplate_300ul\\helmaanalyticsquartz_96_wellplate_300ul (2)', 'Custom Labware\\nanosamplerae

### Step 9: Set up Loaded Dictionary and Directions for Volume Handling
It is important to know how many stock container will be needed for each stock and be able to provide the correct instructions for OT2 commands to follow.

The ranges of stock refers to the ranges of wells one stock will cover. The range is provided in terms of the index of the stock volume dataframe i.e.([lower, upper])). The format is as follows = [[stock1_range1, stock1_range2....], [stock2_range1, stock2_range2....] []...]. 

The range is based on the provided maximum volume for each container, since variabiltiy exist between filling it is **HIGHLY RECCOMENDED** to underestimate the total volume of stock a container can hold so for a 20mL scitillation vials 16000-17000 should be use as the max. Another reason for this is the OT2 will do its best to pick up as much volume as possible but understand liquid in vials can coat the sides not leaving enough depth for the OT2 to grab, so it is **HIGHLY RECCOMENDED** to have at least a **15-20%** volume buffer for each stock

The basis of range is on the maximum volume of the stock container, however currently limitation are: 
 * All stock containers must be the same

In [124]:
importlib.reload(ALH)

<module 'Prepare.OT2Commands' from 'C:\\Users\\lacho\\Documents\\OT2-DOE\\OT2_DOE\\Prepare\\OT2Commands.py'>

In [125]:
protocol = simulate.get_protocol_api('2.8', extra_labware=custom_labware_dict)
loaded_dict = ALH.loading_labware(protocol, plan, well_order='row')

stock_position_info = ALH.stock_well_ranges(complete_df_filtered, loaded_dict['Stock Wells'])
stock_position_info

C:\Users\lacho\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\lacho\.opentrons\deck_calibration.json not found. Loading defaults


{'HEPES-water-stock amount volume uL': {'Ranges': [[0, 16]],
  'Total Volume': 1147.2876228686055,
  'Stock Wells': [A1 of 20mLscintillation_small 12 Well Plate 18000 ÂµL on 1]},
 'Z2M246I-water-stock amount volume uL': {'Ranges': [[0, 16]],
  'Total Volume': 734.3565686337411,
  'Stock Wells': [A2 of 20mLscintillation_small 12 Well Plate 18000 ÂµL on 1]},
 'HAuCl4-water-stock amount volume uL': {'Ranges': [[0, 16]],
  'Total Volume': 173.96147089942747,
  'Stock Wells': [A3 of 20mLscintillation_small 12 Well Plate 18000 ÂµL on 1]},
 'water-stock volume uL': {'Ranges': [[0, 16]],
  'Total Volume': 2744.3943375982262,
  'Stock Wells': [A4 of 20mLscintillation_small 12 Well Plate 18000 ÂµL on 1]}}

In [126]:
# ALH.add_labware_to_dict(loaded_dict, 'Cleaning Wells', ['agilent_6_reservoir_47000ul'], [9])
# ALH.cleaning_tip_protocol(loaded_dict, 2) # can do something like this to add other kwargs prior and after trasnfer

In [138]:
stock_directions = ALH.create_sample_making_directions(complete_df_filtered.drop('HAuCl4-water-stock amount volume uL', axis=1), stock_position_info, loaded_dict, start_position=0)
gold_directions =  ALH.create_sample_making_directions(complete_df_filtered['HAuCl4-water-stock amount volume uL'], stock_position_info, loaded_dict, start_position=0)

In [142]:
ALH.pipette_volumes_component_wise(protocol, stock_directions, loaded_dict, new_tip="once")
ALH.pipette_volumes_component_wise(protocol, gold_directions, loaded_dict, new_tip="always", mix_after=(5, 20))

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 71.70547642928784 from A1 of 20mLscintillation_small 12 Well Plate 18000 ÂµL on 1 to A1 of Corning 96 Well Plate 360 µL Flat on 2
Aspirating 71.70547642928784 uL from A1 of 20mLscintillation_small 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Touching tip
Dispensing 71.70547642928784 uL into A1 of Corning 96 Well Plate 360 µL Flat on 2 at 300.0 uL/sec
Touching tip
Delaying for 0 minutes and 0.0 seconds
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 71.70547642928784 from A1 of 20mLscintillation_small 12 Well Plate 18000 ÂµL on 1 to A1 of Corning 96 Well Plate 360 µL Flat on 2
Aspirating 71.70547642928784 uL from A1 of 20mLscintillation_small 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Dispensing 71.70547642928784 uL into A1 of Corning 96 Well Plate 360 µL Flat on 2 at 300.0 uL/sec
Transferring 71.70547642928784 from A1 of 20mLscintillati

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 71.70547642928784 from A1 of 20mLscintillation_small 12 Well Plate 18000 ÂµL on 1 to A1 of Corning 96 Well Plate 360 µL Flat on 2
Aspirating 71.70547642928784 uL from A1 of 20mLscintillation_small 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Touching tip
Dispensing 71.70547642928784 uL into A1 of Corning 96 Well Plate 360 µL Flat on 2 at 300.0 uL/sec
Touching tip
Delaying for 0 minutes and 0.0 seconds
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 71.70547642928784 from A1 of 20mLscintillation_small 12 Well Plate 18000 ÂµL on 1 to A1 of Corning 96 Well Plate 360 µL Flat on 2
Aspirating 71.70547642928784 uL from A1 of 20mLscintillation_small 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Dispensing 71.70547642928784 uL into A1 of Corning 96 Well Plate 360 µL Flat on 2 at 300.0 uL/sec
Transferring 71.70547642928784 from A1 of 20mLscintillati

### Step 10: Format for Exporting and Upload to Google Drive
* Using the information from our previously created dataframes we create a final dataframe to convert to a csv and upload to google drive for storage. 
    * Currently the two main pieces of information uploaded are composition of sample and sample location information. This for each sample is tied to a unique ID which contains a well_timestamp_keyword. 

* Currently the function *CreateSamples.add_final_location* uses direciton information plus the sampling dataframe either created or imported to add useful information and export as a csv.

In [162]:
export_path = "./test.csv"
export_df = CreateSamples.add_final_location(directions,complete_df_filtered)
# export_df.to_csv(export_path, index = False)
export_df.reset_index(inplace=True, drop=True)

export_df.to_csv(export_path, index=False)